In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp

classrooms = pd.read_csv("../data/processed/classrooms_clean.csv")
courses = pd.read_csv("../data/processed/courses_clean.csv")

print("Optimization notebook loaded")


Optimization notebook loaded


In [2]:
solver = pywraplp.Solver.CreateSolver("SCIP")

if not solver:
    raise Exception("Solver not created")

solver


<ortools.linear_solver.pywraplp.Solver; proxy of <Swig Object of type 'operations_research::MPSolver *' at 0x1117d02a0> >

In [3]:
x = {}

for course in courses.course_id:
    for room in classrooms.classroom_id:
        x[course, room] = solver.BoolVar(f"x_{course}_{room}")

print(f"Decision variables created: {len(x)}")


Decision variables created: 16


In [4]:
for course in courses.course_id:
    solver.Add(
        sum(x[course, room] for room in classrooms.classroom_id) == 1
    )

print("Each course assigned to exactly one classroom")


Each course assigned to exactly one classroom


In [5]:
for course in courses.course_id:
    enrollment = courses.loc[courses.course_id == course, "enrollment"].values[0]
    for room in classrooms.classroom_id:
        capacity = classrooms.loc[classrooms.classroom_id == room, "capacity"].values[0]
        solver.Add(x[course, room] * enrollment <= capacity)

print("Capacity constraints added")


Capacity constraints added


In [6]:
solver.Minimize(0)
status = solver.Solve()
status


0

In [7]:
assignments = []

for course, room in x:
    if x[course, room].solution_value() == 1:
        assignments.append({
            "course": course,
            "classroom": room
        })

pd.DataFrame(assignments)


,course,classroom
0,IE3000,C101
1,IE4000,C101
2,BIO2100,C101
3,BUS3100,C101


In [8]:
distances = pd.read_csv("../data/processed/distances_clean.csv")
distances.head()


,from_location,to_location,minutes
0,Lot51,Engineering,6
1,Lot51,Science,8
2,CentralDeck,Engineering,5
3,CentralDeck,Science,6
4,NorthLot,Business,4


In [9]:
distance_lookup = {}

for _, row in distances.iterrows():
    distance_lookup[(row["from_building"], row["to_building"])] = row["walking_time"]

distance_lookup


KeyError: 'walking_time'

In [10]:
distances.columns


Index(['from_location', 'to_location', 'minutes'], dtype='object')

In [11]:
distances.head()


,from_location,to_location,minutes
0,Lot51,Engineering,6
1,Lot51,Science,8
2,CentralDeck,Engineering,5
3,CentralDeck,Science,6
4,NorthLot,Business,4


In [12]:
distance_lookup = {}

for _, row in distances.iterrows():
    distance_lookup[(row["from_building"], row["to_building"])] = row["distance_minutes"]

distance_lookup


KeyError: 'distance_minutes'

In [13]:
distance_lookup = {}

for _, row in distances.iterrows():
    distance_lookup[(row["from_building"], row["to_building"])] = row["minutes"]

distance_lookup


KeyError: 'from_building'

In [14]:
print(distances.columns.tolist())


['from_location', 'to_location', 'minutes']


In [15]:
distances.head(10)


,from_location,to_location,minutes
0,Lot51,Engineering,6
1,Lot51,Science,8
2,CentralDeck,Engineering,5
3,CentralDeck,Science,6
4,NorthLot,Business,4
5,Engineering,Science,2
6,Engineering,Business,3
7,Science,Business,2


In [16]:
['from', 'to', 'minutes']
ENG → SCI → 6


SyntaxError: invalid character '→' (U+2192) (3349845800.py, line 2)

In [17]:
print(distances.columns.tolist())
distances.head()


['from_location', 'to_location', 'minutes']


,from_location,to_location,minutes
0,Lot51,Engineering,6
1,Lot51,Science,8
2,CentralDeck,Engineering,5
3,CentralDeck,Science,6
4,NorthLot,Business,4


In [18]:
distance_lookup = {}

for _, row in distances.iterrows():
    distance_lookup[(row["from_location"], row["to_location"])] = row["minutes"]

distance_lookup


{('Lot51', 'Engineering'): 6,
 ('Lot51', 'Science'): 8,
 ('CentralDeck', 'Engineering'): 5,
 ('CentralDeck', 'Science'): 6,
 ('NorthLot', 'Business'): 4,
 ('Engineering', 'Science'): 2,
 ('Engineering', 'Business'): 3,
 ('Science', 'Business'): 2}

In [19]:
distances = distances.rename(columns={
    "from_location": "from_building",
    "to_location": "to_building",
    "minutes": "walking_time"
})

distances.head()


,from_building,to_building,walking_time
0,Lot51,Engineering,6
1,Lot51,Science,8
2,CentralDeck,Engineering,5
3,CentralDeck,Science,6
4,NorthLot,Business,4


In [20]:
distance_lookup = {
    (row["from_building"], row["to_building"]): row["walking_time"]
    for _, row in distances.iterrows()
}

distance_lookup


{('Lot51', 'Engineering'): 6,
 ('Lot51', 'Science'): 8,
 ('CentralDeck', 'Engineering'): 5,
 ('CentralDeck', 'Science'): 6,
 ('NorthLot', 'Business'): 4,
 ('Engineering', 'Science'): 2,
 ('Engineering', 'Business'): 3,
 ('Science', 'Business'): 2}

In [21]:
course_departments = dict(zip(courses.course_id, courses.department))
room_buildings = dict(zip(classrooms.classroom_id, classrooms.building))


In [22]:
course_departments = dict(zip(courses.course_id, courses.department))
room_buildings = dict(zip(classrooms.classroom_id, classrooms.building))

course_departments, room_buildings


({'IE3000': 'Engineering',
  'IE4000': 'Engineering',
  'BIO2100': 'Science',
  'BUS3100': 'Business'},
 {'C101': 'Engineering',
  'C102': 'Engineering',
  'C201': 'Science',
  'C301': 'Business'})

In [23]:
objective = solver.Objective()

for course in courses.course_id:
    dept = course_departments[course]
    for room in classrooms.classroom_id:
        building = room_buildings[room]
        walking_time = distance_lookup.get((dept, building), 100)
        objective.SetCoefficient(x[course, room], walking_time)

objective.SetMinimization()
print("Objective function set")


Objective function set


In [24]:
status = solver.Solve()
status


0

In [25]:
optimized = []

for course, room in x:
    if x[course, room].solution_value() == 1:
        optimized.append({
            "course": course,
            "department": course_departments[course],
            "classroom": room,
            "building": room_buildings[room],
            "walking_time_minutes": distance_lookup.get(
                (course_departments[course], room_buildings[room]), None
            )
        })

optimized_df = pd.DataFrame(optimized)
optimized_df


,course,department,classroom,building,walking_time_minutes
0,IE3000,Engineering,C201,Science,2.0
1,IE4000,Engineering,C201,Science,2.0
2,BIO2100,Science,C301,Business,2.0
3,BUS3100,Business,C101,Engineering,NaN


In [26]:
total_walking_time = optimized_df["walking_time_minutes"].sum()
total_walking_time


np.float64(6.0)

In [27]:
optimized_df.to_csv("../outputs/optimized_assignments.csv", index=False)

with open("../outputs/summary.txt", "w") as f:
    f.write(f"Total optimized walking time (minutes): {total_walking_time}")

print("Optimization results saved")


Optimization results saved
